## The Problem - Strategic Asset Production Plan
The new demand forecast and projected orders for the next 5 years just arrived. In order to ensure our companies success, we need to assess the capability of our existing supply chain to meet the forecasted demand.

If we are able to see any issues with the given production plan, we need to come up with mitigating actions in order to ensure we can supply our end customers reliably throughout the entire time horizon.

In order to run this analysis, you are given these 5 data sets: asset_uptime.json, asset_rates.json, skus.json, orders.json, allocation_plan.json


In [1]:
import os
import json
import pandas as pd
import altair as alt
import pulp
import numpy as np

##### Import data from JSON files and convert to pandas

In [2]:
# Checking to ensure json files are in current directory
obj = os.scandir('.')
files = [item.name for item in obj if '.json' in item.name]
print(files)

['allocation_plan.json', 'asset_rates.json', 'asset_uptime.json', 'orders.json', 'skus.json']


In [ ]:
def import_data(file_name: str):
    try:
        with open(file_name, 'r') as file:
            data = json.load(file)
        return pd.DataFrame(data)
    
    except FileNotFoundError:
        print(f"Error: The file '{file_name}' was not found.")

In [ ]:
allocation_plan = import_data('allocation_plan.json')
asset_rates = import_data('asset_rates.json')
asset_uptime = import_data('asset_uptime.json')
orders = import_data('orders.json')
skus = import_data('skus.json')

#### Explore Data Set

In [ ]:
asset_rates.head()

In [ ]:
print(orders.shape[0])
orders.head()

In [ ]:
orders['Date'] = pd.to_datetime(orders[['Year', 'Month']].assign(day=1))

In [ ]:
chart = alt.Chart(orders).mark_line(point=True).encode(
    alt.X('Date', timeUnit='yearquarter', title = 'Year'),  # change time unit to year for less granularity
    alt.Y('sum(Demand):Q', title = 'Aggregate Demand in Units'), # Aggregate demand by SKU
    color='SKU',
).properties(width=1200, height=500,  title = 'Projected Demand over time')

chart

#### Calculate Capactiy

In [ ]:
# Calulating asset working hours per year
asset_uptime['hours_per_year'] = (asset_uptime['days_per_week'] * asset_uptime['weeks_per_year'] * asset_uptime['hours_per_shift'] * asset_uptime['shifts_per_day'])
asset_uptime

Calculate how long it takes each asset to produce one batch - assumption that must produce products in batches not by number of units\
run_rate = units/hour, lot_size = units/batch\
hr/unit * unit/batch = hr/batch\
**(lot_size / run_rate) + cleanup_time = hr/batch**

In [ ]:
asset_rates_merged = asset_rates.merge(skus, on='Product', how='left')
asset_rates_merged['hours_per_batch'] = ((asset_rates_merged['Lot Size'] / asset_rates_merged['run_rate']) + asset_rates_merged['cleanup_time'])
asset_rates_merged.head()

#### Calculate Demand

In [ ]:
demand_merged = orders.merge(allocation_plan, on='proj_id', how='left')
demand_merged = demand_merged.rename(columns = {'Asset' : 'Asset Allocated'})
demand_merged.head()

In [ ]:
#calculate annual demand by asset (as granularity requested)
demand_by_asset = demand_merged.groupby(['Year', 'SKU', 'Asset Allocated'])['Demand'].sum().reset_index()
demand_by_asset.head(6)

In [ ]:
demand_by_asset = demand_by_asset.merge(asset_rates_merged, left_on=['SKU', 'Asset Allocated'], right_on=['Product', 'asset_id'], how='left').drop(columns=['Product', 'asset_id'])

In [ ]:
# Calculate batches
def calculate_batches(row):
    batches = (row['Demand']//row['Lot Size']) + (row['Demand'] % row['Lot Size'] > 0)
    return batches
demand_by_asset['Batches'] = demand_by_asset.apply(calculate_batches, axis=1)

In [ ]:
# Calculate how many hours are needed for each SKU
def aggregate_hrs_per_sku(row):
    hours = row['Batches']* row['hours_per_batch']
    return hours
demand_by_asset['Hours Required'] = demand_by_asset.apply(aggregate_hrs_per_sku, axis=1)


In [ ]:
demand_by_asset.head()

#### Assess Plan

In [ ]:
asset_utilization = demand_by_asset.groupby([ 'Asset Allocated', 'Year'])['Hours Required'].sum().reset_index()
extracted = asset_uptime.loc[:, ['hours_per_year', 'asset_id']]
asset_utilization = asset_utilization.merge(extracted, left_on='Asset Allocated', right_on='asset_id').drop(columns='asset_id')
asset_utilization['Utilization Ratio'] = asset_utilization['Hours Required'] / asset_utilization['hours_per_year']


In [ ]:
asset_utilization[asset_utilization['Utilization Ratio'] >=1] # oh no.....

In [ ]:

Utilization_chart =  alt.Chart(asset_utilization).mark_line().encode(
        alt.X('Year:O', title='Year' ),
        y = alt.Y('Utilization Ratio:Q', title = 'Utilization Ratio'),
        color = 'Asset Allocated').properties(width = 500, title = 'Asset Capacity Utilization by Asset and Year')

over_utilized = pd.DataFrame({'y': [1.0]})
horizontal_line = alt.Chart(over_utilized).mark_rule(color='black', strokeDash=[5, 5]).encode(
    y = alt.Y('y:Q'))

Utilization_chart + horizontal_line

# can multiply ratio by 100 to speak to percentages for business presentations

#### Explore a Solution

In [ ]:
# finding the most efficient assets for each SKU
efficient_indices = asset_rates_merged.groupby('Product')['hours_per_batch'].idxmin()
efficient_assets  = asset_rates_merged.loc[efficient_indices]
efficient_assets = efficient_assets.reset_index(drop=True)
efficient_assets

In [ ]:
# finding the most efficient product for each asset
efficient_indices = asset_rates_merged.groupby('asset_id')['hours_per_batch'].idxmin()
efficient_products = asset_rates_merged.loc[efficient_indices]
efficient_products = efficient_products.reset_index(drop=True)
efficient_products

In [ ]:
demand_by_product = orders.groupby(['Year', 'SKU'])['Demand'].sum().drop(columns = ['proj_id', 'Date']).reset_index()
demand_by_product.head(6)

Mixed Integer Linear Programming\
'Branch and Bound' Algorithm

In [ ]:
demand_map = demand_by_product.set_index(['SKU', 'Year'])['Demand'].to_dict()
capacity_map = asset_uptime.set_index('asset_id')['hours_per_year'].to_dict()

# C. Create Sets (Indexes)
products = skus['Product'].unique()
assets = asset_uptime['asset_id'].unique()
years = demand_by_product['Year'].unique() 

# Create mapping dictionaries for remaining lookups
lot_size_map = skus.set_index('Product')['Lot Size'].to_dict()
hours_per_batch_map = asset_rates_merged.set_index(['asset_id', 'Product'])['hours_per_batch'].to_dict()

# --- 2. OPTIMIZATION PROBLEM SETUP ---
model = pulp.LpProblem("Asset_Allocation", pulp.LpMinimize)

# Define Decision Variables
X = pulp.LpVariable.dicts("Units", (products, assets, years), lowBound=0, cat='Continuous')  # keep as continuous so that it is easier to solve
B = pulp.LpVariable.dicts("Batches", (products, assets, years), lowBound=0, cat='Integer')

# Objective Function: Minimize Total Production Time (Hours)
model += (
    pulp.lpSum([
        B[i][j][y] * hours_per_batch_map.get((j, i), 0)
        for i in products for j in assets for y in years
    ]), "Total_Production_Hours"
)

# --- 3. CONSTRAINTS ---

# A. Demand Constraint (Annual): Must meet all annual demand
for i in products:
    for y in years:
        demand = demand_map.get((i, y), 0)
        model += (
            pulp.lpSum([X[i][j][y] for j in assets]) == demand,
            f"Demand_Met_{i}_{y}"
        )

# B. Capacity Constraint (Annual): Cannot exceed annual capacity on any asset
for j in assets:
    capacity = capacity_map.get(j, 0)
    for y in years:
        model += (
            pulp.lpSum([
                B[i][j][y] * hours_per_batch_map.get((j, i), 0)
                for i in products if (j, i) in hours_per_batch_map
            ]) <= capacity,
            f"Capacity_Limit_{j}_{y}"
        )

# C. Batch Linking Constraint: Forces sufficient integer batches for units produced
for i in products:
    lot_size = lot_size_map.get(i, np.inf) 
    if lot_size <= 0 or lot_size == np.inf:
        continue
        
    for j in assets:
        for y in years:
            model += (
                lot_size * B[i][j][y] >= X[i][j][y],
                f"Batch_Link_{i}_{j}_{y}"
            )

# --- 4. SOLVE AND RETURN RESULTS ---

# Use the default solver (usually CBC)
model.solve() 

if model.status == pulp.LpStatusOptimal:
    print("Status: Optimal annual solution found.")
    
    # ... (Code to extract and format results into a DataFrame, similar to previous function)
    results = []
    for i in products:
        for j in assets:
            for y in years:
                units = X[i][j][y].varValue
                if units > 0.0001:
                    batches = B[i][j][y].varValue
                    hours_required = batches * hours_per_batch_map.get((j, i), 0)
                    
                    results.append({
                        'Asset': j,
                        'Product': i,
                        'Year': y,
                        'Units_Allocated': units,
                        'Batches_Run': batches,
                        'Hours_Required': hours_required
                    })
                    
    df_results = pd.DataFrame(results)
    total_hours = pulp.value(model.objective)
    
    
else:
    print(f"Status: Solver stopped with status code {pulp.LpStatus[model.status]}.")
    if model.status == pulp.LpStatusInfeasible:
        print("The annual demand exceeds the annual capacity. Cannot meet all production targets.")
    
    

In [ ]:
df_results['Asset'].unique() # looks as though L042 and L007 are not only most efficient but are sufficient for all demand through 2030.... 
                             # consider operations cost of other assets, locations and cost of transportation from assets, 
                             # as well as expansion and reserve plans, and costs of shutting down vs remaining operational


Validate Results

In [ ]:
df_results[df_results['Year'] == 2027]

In [ ]:
demand_by_product[demand_by_product['Year'] == 2027]

In [ ]:
# very rough checks for total demand and total hours across all assets
if (int(df_results['Units_Allocated'].sum())) != int(demand_by_product['Demand'].sum()): print('ERROR: demand not met')
else: print('Demand met')
for year in years:
    if (int(df_results[df_results['Year'] == year]['Hours_Required'].sum())) > int(asset_uptime['hours_per_year'].sum()): 
        print(f'ERROR: hours error for year {year}')

Visualize Results

In [ ]:
# prepare new utilization ratios
asset_utilization = df_results.groupby([ 'Asset', 'Year'])['Hours_Required'].sum().reset_index()
extracted = asset_uptime.loc[:, ['hours_per_year', 'asset_id']]
asset_utilization = asset_utilization.merge(extracted, left_on='Asset', right_on='asset_id').drop(columns='asset_id')
asset_utilization['Utilization Ratio'] = asset_utilization['Hours_Required'] / asset_utilization['hours_per_year']
asset_utilization


In [ ]:

Utilization_chart =  alt.Chart(asset_utilization).mark_line().encode(
        alt.X('Year:O', title='Year' ),
        y = alt.Y('Utilization Ratio:Q', title = 'Utilization Ratio'),
        color = 'Asset').properties(width = 500, title = 'Suggested Asset Capacity Utilization by Asset and Year')

over_utilized = pd.DataFrame({'y': [1.0]})
horizontal_line = alt.Chart(over_utilized).mark_rule(color='black', strokeDash=[5, 5]).encode(
    y = alt.Y('y:Q',  scale=alt.Scale(domain=[0, 1.1])))

Utilization_chart + horizontal_line

In [ ]:
# CONSIDERATIONS
# dont know if you have excess product in batches if you will retain stock
# assumption 1 you do not have abilitiy to reserve stock - ideally avoid over production
# ensure that you maximize which asset is producing which units - want to produce a SKU at the most efficient assets for the SKU 
# changes in holiday/seasonality
